In [1]:
# IMPORTS

import platform
import sys
import time
from shutil import copyfile
from tempfile import TemporaryDirectory

import numpy
from sklearn.datasets import load_breast_cancer

from concrete.ml.deployment import FHEModelClient, FHEModelDev, FHEModelServer
from concrete.ml.sklearn import XGBClassifier

In [2]:
# ENVIROMENT CLASS

class OnDiskenviroment:
    def __init__(self):
        self.server_dir = TemporaryDirectory()  
        self.client_dir = TemporaryDirectory()
        self.dev_dir = TemporaryDirectory()

    def client_send_evaluation_key_to_server(self, serialized_evaluation_keys):
        with open(self.server_dir.name + "/serialized_evaluation_keys.ekl", "wb") as f:
            f.write(serialized_evaluation_keys)

    def client_send_input_to_server_for_prediction(self, input_after_encryption):
        with open(self.server_dir.name + "/serialized_evaluation_keys.ekl", "rb") as f:
            serialized_evaluation_keys = f.read()
        time_begin = time.time()
        encrypted_prediction = FHEModelServer(self.server_dir.name).run(
            input_after_encryption, serialized_evaluation_keys
        )
        time_end = time.time()
        with open(self.server_dir.name + "/encrypted_prediction.enc", "wb") as f:
            f.write(encrypted_prediction)
        return time_end - time_begin

    def dev_send_model_to_server(self):
        copyfile(self.dev_dir.name + "/server.zip", self.server_dir.name + "/server.zip")

    def server_send_encrypted_prediction_to_client(self):
        with open(self.server_dir.name + "/encrypted_prediction.enc", "rb") as f:
            encrypted_prediction = f.read()
        return encrypted_prediction

    def dev_send_clientspecs_and_modelspecs_to_client(self):
        copyfile(self.dev_dir.name + "/client.zip", self.client_dir.name + "/client.zip")
        copyfile(
            self.dev_dir.name + "/serialized_processing.json",
            self.client_dir.name + "/serialized_processing.json",
        )

    def cleanup(self):
        self.server_dir.cleanup()
        self.client_dir.cleanup()
        self.dev_dir.cleanup()

In [3]:
# MODEL PREPARATION TO FHE AND ENVIROMENT

X, y = load_breast_cancer(return_X_y=True)

X_model_owner, X_client = X, X
y_model_owner, y_client = y, y

n_estimators = 10

model_to_fhe = XGBClassifier(n_bits=2, n_estimators=n_estimators, max_depth=3)
model_to_fhe.fit(X_model_owner, y_model_owner)
model_to_fhe.compile(X_model_owner)

enviroment = OnDiskenviroment()

fhemodel_to_fhe = FHEModelDev(enviroment.dev_dir.name, model_to_fhe)
fhemodel_to_fhe.save()


In [4]:
# SENDING APPROPRIATE DATA TO ENVIROMENT 

!ls -lh $enviroment.dev_dir.name
enviroment.dev_send_model_to_server()
!ls -lh $enviroment.server_dir.name
enviroment.dev_send_clientspecs_and_modelspecs_to_client()
!ls -lh $enviroment.client_dir.name

total 60K
-rw-rw-r-- 1 cybant cybant  464 lis 15 23:02 client.zip
-rw-rw-r-- 1 cybant cybant  44K lis 15 23:02 serialized_processing.json
-rw-rw-r-- 1 cybant cybant 8,3K lis 15 23:02 server.zip
total 12K
-rw-rw-r-- 1 cybant cybant 8,3K lis 15 23:03 server.zip
total 48K
-rw-rw-r-- 1 cybant cybant 464 lis 15 23:03 client.zip
-rw-rw-r-- 1 cybant cybant 44K lis 15 23:03 serialized_processing.json


In [5]:
# CLIENT PREPERATION - KEYS

client_model = FHEModelClient(enviroment.client_dir.name, key_dir=enviroment.client_dir.name)
client_model.generate_private_and_evaluation_keys()

serialized_evaluation_keys = client_model.get_serialized_evaluation_keys()

print(f"Evaluation keys size: {sys.getsizeof(serialized_evaluation_keys) / 1024 / 1024:.2f} MB")
enviroment.client_send_evaluation_key_to_server(serialized_evaluation_keys)

Evaluation keys size: 119.88 MB


KeySetCache: miss, regenerating /tmp/tmpaidg6p5l/7071120041192343400/0_0


In [6]:
# MAIN LOOP WHICH ENRYPT DATA AND MAKE STATISTICS

decrypted_predictions = []
time_exec = []
for i in range(X_client.shape[0]):
    if i % 20 == 0:
        print("Step " + str(i) + " done")
    buffor = X_client[[i], :]
    input_after_encryption = client_model.quantize_encrypt_serialize(buffor)
    time_exec += [enviroment.client_send_input_to_server_for_prediction(input_after_encryption)]
    encrypted_prediction = enviroment.server_send_encrypted_prediction_to_client()
    decrypted_prediction = client_model.deserialize_decrypt_dequantize(encrypted_prediction)[0]
    decrypted_predictions.append(decrypted_prediction)

clear_prediction_classes = model_to_fhe.predict_proba(X_client).argmax(axis=1)
decrypted_predictions_classes = numpy.array(decrypted_predictions).argmax(axis=1)
accuracy = (clear_prediction_classes == decrypted_predictions_classes).mean()



Step 0 done
Step 20 done
Step 40 done
Step 60 done
Step 80 done
Step 100 done
Step 120 done
Step 140 done
Step 160 done
Step 180 done
Step 200 done
Step 220 done
Step 240 done
Step 260 done
Step 280 done
Step 300 done
Step 320 done
Step 340 done
Step 360 done
Step 380 done
Step 400 done
Step 420 done
Step 440 done
Step 460 done
Step 480 done
Step 500 done
Step 520 done
Step 540 done
Step 560 done


In [7]:
# INTERESTING PRINTOUTS
print(f"Encrypted data is "f"{sys.getsizeof(input_after_encryption)/sys.getsizeof(buffor):.2f}"" larger than data")
print(f"Execution time are {[numpy.round(e, 2) for e in time_exec]}, ie an average of " f"{numpy.mean(time_exec):.2f} seconds")
print(f"Accuracy between FHE prediction and clear model is: {accuracy*100:.0f}%")

Encrypted data is 1336.48 larger than data
Execution time are [5.24, 4.4, 4.2, 5.1, 5.77, 5.39, 5.24, 5.19, 5.07, 5.1, 5.03, 5.1, 5.16, 5.02, 5.08, 5.0, 5.08, 5.0, 5.1, 5.03, 5.07, 4.99, 5.1, 5.05, 5.0, 5.05, 5.37, 5.77, 5.01, 5.02, 5.01, 5.01, 4.99, 5.03, 4.97, 4.97, 4.97, 4.99, 5.01, 4.99, 4.98, 4.98, 4.98, 4.98, 5.0, 5.0, 4.99, 4.97, 4.98, 5.17, 5.42, 5.22, 6.76, 5.89, 5.69, 6.93, 6.2, 5.58, 6.85, 8.03, 6.48, 7.97, 8.52, 6.29, 6.26, 6.13, 5.8, 5.92, 6.39, 5.73, 5.67, 5.71, 7.21, 6.08, 5.94, 5.91, 6.35, 5.69, 5.88, 5.85, 5.93, 5.92, 5.58, 5.5, 5.49, 5.54, 5.51, 5.45, 5.46, 5.49, 5.47, 5.5, 5.48, 5.52, 5.5, 5.52, 5.37, 5.46, 5.48, 5.38, 5.49, 5.72, 5.54, 5.37, 5.4, 5.39, 5.46, 5.52, 5.55, 5.51, 5.47, 5.4, 5.39, 5.35, 5.39, 5.55, 5.43, 5.44, 5.43, 5.41, 5.52, 5.76, 5.51, 5.51, 5.49, 5.51, 5.46, 5.47, 5.4, 5.68, 5.54, 5.47, 5.44, 5.59, 5.42, 5.52, 5.48, 5.39, 5.43, 5.42, 5.68, 5.57, 5.36, 5.27, 5.35, 5.49, 5.36, 5.32, 5.31, 5.25, 5.38, 5.24, 5.26, 5.16, 5.31, 5.19, 5.15, 5.34, 5.35, 5.2